In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import requests
import osmnx as ox
import shapely as shp
from ast import literal_eval
from pyproj import Geod
import matplotlib.pyplot as plt
import folium

In [2]:
# somehwere in manhattan
loc = (40.73188892751728, -73.98927188478527)

In [3]:
df = pd.read_csv("data/Institutional_Provider_Network_Data__2018_Quarter_4.csv")

C:\Users\yianz\AppData\Local\Temp\ipykernel_28264\2616766193.py:1: DtypeWarning: Columns (1,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/Institutional_Provider_Network_Data__2018_Quarter_4.csv")


In [4]:
# lots of entries
df_ny = df[df["State_std"] == "NY"]
df_ny

,SiteName_std,roomno,Address1_std,City_std,TrueCountyFIPS,TrueCountyName,State_std,Zip_std,Latitude,Longitude,...,hx_shp_med_ind_9,hx_shp_med_ind_10,ServCnty1,ServCnty2,ServCnty3,ServCnty4,ServCnty5,PlanName,PNDSID,Quarter
0,A+ MEDITRANS,NaN,23 RAILROAD AVE,ALBANY,36001.0,Albany,NY,12205.0,42.721239,-73.855800,...,0,0,999,999,999,999,999,Eddy Senior Care,28,Quarter4 2018
1,ACCENT HEALTH CARE SERVICES INC,NaN,820 5TH AVE,TROY,36083.0,Rensselaer,NY,12182.0,42.783162,-73.669198,...,0,0,83,93,999,999,999,Eddy Senior Care,28,Quarter4 2018
2,ALBANY COUNTY NURSING HOME,NaN,780 ALBANY SHAKER RD,ALBANY,36001.0,Albany,NY,12211.0,42.744227,-73.813865,...,0,0,999,999,999,999,999,Eddy Senior Care,28,Quarter4 2018
3,ALBANY REGIONAL EYE SURGERY CENTER,Suite 2,5 JOHNSON RD,LATHAM,36001.0,Albany,NY,12110.0,42.762532,-73.748559,...,0,0,999,999,999,999,999,Eddy Senior Care,28,Quarter4 2018
4,AVEENAS LOVING COMPANION CARE,NaN,9115 181ST ST,HOLLIS,36081.0,Queens,NY,11423.0,40.709408,-73.779800,...,0,0,999,999,999,999,999,Eddy Senior Care,28,Quarter4 2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706521,WYOMING COUNTY COMMUNITY HOSPITAL,NaN,400 N MAIN ST,WARSAW,36121.0,Wyoming,NY,14569.0,42.754333,-78.131197,...,0,0,999,999,999,999,999,Consolidated Health Plans: MagnaCare,169,Quarter4 2018
706522,BRONXCARE DR MARTIN LUTHER KING JR,NaN,1067 OGDEN AVE,BRONX,36005.0,Bronx,NY,10452.0,40.835026,-73.928535,...,0,0,999,999,999,999,999,Consolidated Health Plans: MagnaCare,169,Quarter4 2018
706523,BRONXCARE DR MARTIN LUTHER KING JR,NaN,1265 FRANKLIN AVE,BRONX,36005.0,Bronx,NY,10456.0,40.831187,-73.902746,...,0,0,999,999,999,999,999,Consolidated Health Plans: MagnaCare,169,Quarter4 2018
706524,BRONXCARE DR MARTIN LUTHER KING JR,NaN,1591 FULTON AVE,BRONX,36005.0,Bronx,NY,10457.0,40.838989,-73.899620,...,0,0,999,999,999,999,999,Consolidated Health Plans: MagnaCare,169,Quarter4 2018


In [5]:
df_ny["geometry"] = df_ny.apply(lambda x: shp.Point((x["Longitude"], x["Latitude"])), axis=1)

C:\Users\yianz\AppData\Local\Temp\ipykernel_28264\3728899583.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ny["geometry"] = df_ny.apply(lambda x: shp.Point((x["Longitude"], x["Latitude"])), axis=1)


In [6]:
# filter out only nyc hospoitals
geod = Geod(ellps="WGS84")

In [7]:
df_ny["distance_from_nyc"] = df_ny["geometry"].apply(lambda x: geod.geometry_length(shp.geometry.LineString([x, shp.Point(loc[::-1])])))
df_ny["distance_from_nyc"].describe()

C:\Users\yianz\AppData\Local\Temp\ipykernel_28264\1844202211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ny["distance_from_nyc"] = df_ny["geometry"].apply(lambda x: geod.geometry_length(shp.geometry.LineString([x, shp.Point(loc[::-1])])))


count    6.492230e+05
mean     1.198448e+05
std      1.965159e+05
min      9.689857e+01
25%      1.202399e+04
50%      2.624854e+04
75%      2.201840e+05
max      4.348459e+06
Name: distance_from_nyc, dtype: float64

In [8]:
df_nyc = df_ny[df_ny["distance_from_nyc"] <= 32000]
df_nyc

,SiteName_std,roomno,Address1_std,City_std,TrueCountyFIPS,TrueCountyName,State_std,Zip_std,Latitude,Longitude,...,ServCnty1,ServCnty2,ServCnty3,ServCnty4,ServCnty5,PlanName,PNDSID,Quarter,geometry,distance_from_nyc
4,AVEENAS LOVING COMPANION CARE,NaN,9115 181ST ST,HOLLIS,36081.0,Queens,NY,11423.0,40.709408,-73.779800,...,999,999,999,999,999,Eddy Senior Care,28,Quarter4 2018,POINT (-73.7798 40.709408),17873.357494
92,ARMS ACRES INC,NaN,3584 JEROME AVE,BRONX,36005.0,Bronx,NY,10467.0,40.884972,-73.879143,...,999,999,999,999,999,Beacon Health Options: Empire Plan,15,Quarter4 2018,POINT (-73.879143 40.884972),19373.874657
93,ARMS ACRES INC,NaN,3584 JEROME AVE,BRONX,36005.0,Bronx,NY,10467.0,40.884972,-73.879143,...,999,999,999,999,999,Beacon Health Options: Empire Plan,15,Quarter4 2018,POINT (-73.879143 40.884972),19373.874657
94,ARMS ACRES INC,NaN,3584 JEROME AVE,BRONX,36005.0,Bronx,NY,10467.0,40.884972,-73.879143,...,999,999,999,999,999,Beacon Health Options: Empire Plan,15,Quarter4 2018,POINT (-73.879143 40.884972),19373.874657
95,ARMS ACRES INC,NaN,3584 JEROME AVE,BRONX,36005.0,Bronx,NY,10467.0,40.884972,-73.879143,...,999,999,999,999,999,Beacon Health Options: Empire Plan,15,Quarter4 2018,POINT (-73.879143 40.884972),19373.874657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706520,WYCKOFF HEIGHTS MEDICAL CENTER,NaN,374 STOCKHOLM ST,BROOKLYN,36047.0,Kings,NY,11237.0,40.704119,-73.917728,...,999,999,999,999,999,Consolidated Health Plans: MagnaCare,169,Quarter4 2018,POINT (-73.917728 40.704119),6786.100440
706522,BRONXCARE DR MARTIN LUTHER KING JR,NaN,1067 OGDEN AVE,BRONX,36005.0,Bronx,NY,10452.0,40.835026,-73.928535,...,999,999,999,999,999,Consolidated Health Plans: MagnaCare,169,Quarter4 2018,POINT (-73.928535 40.835026),12548.430095
706523,BRONXCARE DR MARTIN LUTHER KING JR,NaN,1265 FRANKLIN AVE,BRONX,36005.0,Bronx,NY,10456.0,40.831187,-73.902746,...,999,999,999,999,999,Consolidated Health Plans: MagnaCare,169,Quarter4 2018,POINT (-73.902746 40.831187),13226.542142
706524,BRONXCARE DR MARTIN LUTHER KING JR,NaN,1591 FULTON AVE,BRONX,36005.0,Bronx,NY,10457.0,40.838989,-73.899620,...,999,999,999,999,999,Consolidated Health Plans: MagnaCare,169,Quarter4 2018,POINT (-73.89962 40.838989),14096.723210


In [9]:
df_nyc["desser"].value_counts()

760    135712
914     34957
375     16370
599      8811
823      7698
        ...  
14          1
989         1
406         1
984         1
162         1
Name: desser, Length: 233, dtype: int64

In [11]:
for x, y in zip(df_nyc["desser"].value_counts().index, df_nyc["desser"].value_counts().values):
    print(x, y)

760 135712
914 34957
375 16370
599 8811
823 7698
307 7684
749 7015
321 6007
200 5899
660 5480
663 5377
668 4843
11 4557
836 4526
839 4037
837 3852
858 3723
911 3404
17 3309
855 3309
680 3140
860 3109
854 2895
824 2833
862 2822
861 2793
671 2678
913 2580
18 2181
317 1965
662 1855
829 1793
250 1746
665 1721
859 1714
318 1553
919 1480
664 1310
992 1309
856 1285
816 1280
999 1163
827 1152
969 1144
993 1095
302 984
857 908
971 891
831 886
670 855
830 719
300 664
834 629
319 599
828 578
81 539
928 533
326 513
61 478
669 463
936 419
405 372
964 356
5 356
778 342
314 335
974 282
833 281
822 280
832 256
615 225
301 219
59 208
667 206
192 200
312 185
931 175
666 169
661 163
640 156
674 154
817 145
906 142
798 123
953 121
616 112
958 111
819 106
781 105
672 101
982 99
799 93
675 89
909 86
818 85
820 83
191 80
356 77
135 77
20 76
924 75
329 74
201 74
310 73
780 72
85 69
930 66
308 65
907 64
956 64
775 63
315 62
491 62
944 58
929 55
954 54
738 50
790 48
210 47
283 45
979 44
825 42
43 41
996 41
673 

In [12]:
for x in np.unique(df_nyc["SiteName_std"]):
    print(x)

#REF!
1 STOP PHARMACY
1 STOP PHARMACY *
1 WISE PODIATRY CARE PC
101ST AVENUE FOOT CARE P.C.
103 PHARMACY
103 PHARMACY INC
103 PHARMACY INC *
103 PHARMACY INC.
1110 PENNSYLVANIA AVENUE MEDICAL PL
1110 PENNSYLVANIA AVENUE MEDICAL PLLC
1110 PENNSYLVANIA AVENUE MEDICAL, PLLC
113TH STREET MEDICAL GROUP PC
1150 WEBSTER PHARMACY
116 STREET DENTAL CARE PLLC
121 CARE
121 CARE/POMAC
123 LEARN TO SPEAK INC.
124 RX CORP.
125TH DENTAL ARTS P.C.
125TH ST BRACES
125TH STREET CLINIC
125TH STREET DENTAL ARTS P.C.
125TH STREET ORTHODONTICS
127-01 METRO MANAGEMENT CORP
133RD ST PHARMACY INC
133RD ST PHARMACY INC.
133RD STREET PHARMACY
139 CENTER PHARMACY
139 CENTER PHARMACY  LLC
139 CENTER PHARMACY LLC
139 CENTER PHARMACY, LLC
139 CENTER PHARMACY, LLC *
139 PHARMACY
139 PHARMACY INC
139 PHARMACY INC *
139 PHARMACY INC.
13TH AVE PHARMACY
13TH AVE PHARMACY INC
13TH AVE PHARMACY INC.
13TH AVE PHARMACY INC. *
13TH AVENUE PHARMACY INC.
14 STREET MEDICAL RADIOLOGY
1413 RX
1413 RX CORP
1413 RX CORP *
1413 RX CO

## Notes



benefits over health facility genearl information:

- more complete (e.g., has urgent care)
- have more in-depth segmentation (instead of clinic, have type of clinic and what it actually does, which can inform whether a facility supports walk-ins), (desser + service types)
    - e.g., hospital extension clinics in hfgi include a methadone center, a dance injury center, and a dentistry center while in pnds, they are segmented into more identifying categories
- includes the number of providers (noprov)

cons versus health facility general information

- more complex (has a bunch of random, irrelevant stuff)
- indexed by health facilities & plans based on which plans contract with what facilities -- not directly comparable to healthcare accessibility without some tinkering

documentation: https://www.health.ny.gov/health_care/managed_care/docs/dictionary.pdf

what columns mean: https://www.opendatanetwork.com/dataset/health.data.ny.gov/eary-k6ia

To use this data, we need to assume the following:

1. The institutions that are involved in group medical insurance plans are representative of all relevant institutions.
    - this is somewhat reasonable since these institutions would be the most accessible to the average person
2. The assigned designated service is representative of what a facility actually does and can be used to determine whether a facility supports walk-ins or not
    - (can't guarantee that walk-ins are supported but can make generalizations like that clinics generally allow walk-ins)
3. Some subset of the data can be validly excluded, as it is not relevant to medical accessibility
    - should ideally exclude all medically tangential businesses while including relatively relevant ones (like urgent care)

Potential service codes to look for (that might support walk-ins)

- 005 state operated clinic
- 011 general hospital
- 034 state operated facility
- 150 pediatrics
- 252 primary care initiative in underserved areas
- 760 pharmacy?
- 776 general mpractice only - nospec?
- 777 all physician?
- 823 urgent care
- 899 hospital inpatient?
- 901 emergency room
- 914 general medicine - clinic speciality ?
- 978 preferred primary care clinic?
- clinic specialty things (have basically one for every field)